In [ ]:
# Hybrid search over Images in S3 bucket
hybrid_search_query = """WITH filtered_products AS (
            -- First get all men's products
            SELECT img_id, productdisplayname
            FROM products 
            WHERE gender = '{selected_gender}'
        )
        SELECT 
            result.key as id,
            fp.productdisplayname as description,
            result.distance as score
        FROM filtered_products fp
        CROSS JOIN LATERAL aidb.retrieve_key('{st.session_state.text_retriever_name}', '{text_query}', 40) AS result
        WHERE result.key = CONCAT(fp.img_id, '.jpg')
        ORDER BY score ASC LIMIT 5;"""

In [2]:
import pandas as pd
import psycopg2
import time
import os

from io import StringIO
from PIL import Image
# from transformers import CLIPModel, CLIPProcessor, AutoTokenizer, AutoModel
import sys
# Add the parent directory of 'code' to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

print("Python search paths:", sys.path)  # Debugging line
from utils.db_connection import create_db_connection
from utils.generate_embeddings import generate_ollama_embeddings

Python search paths: ['/Users/bilge.ince/.pyenv/versions/3.10.4/lib/python310.zip', '/Users/bilge.ince/.pyenv/versions/3.10.4/lib/python3.10', '/Users/bilge.ince/.pyenv/versions/3.10.4/lib/python3.10/lib-dynload', '', '/Users/bilge.ince/.local/lib/python3.10/site-packages', '/Users/bilge.ince/.pyenv/versions/3.10.4/lib/python3.10/site-packages', '/Users/bilge.ince/projects/hybrid-recommendation-engine']


In [ ]:
conn = create_db_connection()

In [22]:
text_embeddings = generate_ollama_embeddings("summer outfit for thailand")
# 10.5s

In [ ]:
with conn.cursor() as cursor:
    cursor.execute(
        f"""EXPLAIN ANALYSE SELECT img_id, 1-(embedding <=> '{text_embeddings}') AS score FROM products_embeddings_ollama ORDER BY embedding <=> '{text_embeddings}' LIMIT 10;"""
    )  # Create the database
    # 13.2 s
    cursor.fetchall()

[(43666, 0.45893446733961896),
 (13510, 0.4566795844783741),
 (39720, 0.4514953570806024),
 (43655, 0.4487756464496022),
 (30139, 0.44834972554618724),
 (30142, 0.44834972554618724),
 (30146, 0.44729700056278787),
 (30141, 0.44729700056278787),
 (22666, 0.44379710463770283),
 (24845, 0.4427767332231144)]

In [18]:
cursor.execute("""CREATE TABLE IF NOT EXISTS products_embeddings_pgvector (img_id INTEGER PRIMARY KEY REFERENCES products(img_id) ON DELETE CASCADE,
            embedding vector(4096));""")

In [19]:
cursor.execute("""CREATE INDEX ON products_embeddings_pgvector USING hnsw (embedding vector_l2_ops)
WITH (m = 16, ef_construction = 64);""")

InternalError_: column cannot have more than 2000 dimensions for hnsw index


# Benchmarking Pgvector vs Vchord

In [3]:
conn = create_db_connection() # Connect to the database
conn.autocommit = True  # Enable autocommit for creating the database
cursor = conn.cursor()

## Pgvector

In [27]:
cursor.execute("""
        CREATE TABLE IF NOT EXISTS benchmark_embeddings_pgvec(
            img_id INTEGER PRIMARY KEY REFERENCES products(img_id) ON DELETE CASCADE,
            embedding vector(384));""")



cursor.execute("""CREATE INDEX ON benchmark_embeddings_pgvec USING hnsw (embedding vector_l2_ops)
WITH (m = 16, ef_construction = 64);""")



In [4]:
import time

from transformers import AutoTokenizer, AutoModel
# Load the model and processor
text_tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
text_model = AutoModel.from_pretrained("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

## Embedding Generation in batches

In [ ]:
# Initialize timing variables for overall function performance tracking
function_start_time = time.time()

# Fetch data from the database
fetch_start = time.time()
cursor = conn.cursor()
cursor.execute("SELECT img_id, productdisplayname FROM products;")
result = cursor.fetchall()
fetch_end = time.time()

batch_size = 1000
total_rows_inserted = 0

for i in range(0, len(result), batch_size):
    batch_data = [(row[0], row[1]) for row in result[i:i+batch_size] if row[1]]
    if not batch_data:
        continue

    batch_ids, batch_texts = zip(*batch_data)
    batch_texts = [text for text in batch_texts if text]

    if not batch_texts:
        continue

    text_inputs = text_tokenizer(list(batch_texts), return_tensors="pt", padding=True, truncation=True)
    text_model_output = text_model(**text_inputs)
    text_embeddings = text_model_output.last_hidden_state.mean(dim=1).squeeze().detach().cpu().numpy().tolist()

    with conn.cursor() as cursor:
        for idx, embedding in enumerate(text_embeddings):
            cursor.execute(
                "INSERT INTO benchmark_embeddings_pgvec (img_id, embedding) VALUES (%s, %s)",
                (batch_ids[idx], embedding)
            )
            total_rows_inserted += 1

function_end_time = time.time()
total_time = function_end_time - function_start_time

print(f"Total Rows: {total_rows_inserted}")
print(f"Total function execution time: {total_time} seconds")
print(f"Fetching time: {fetch_end - fetch_start} seconds")
# Total Rows: 44433
# Total function execution time: 309.5544641017914 seconds
# Embedding Insertion time: 308.15446400642395 seconds ~ 5 minutes
# Model loading time: 1.4021379947662354 seconds
# Fetching time: 0.04068493843078613 seconds


Total Rows: 44433
Total function execution time: 309.5544641017914 seconds
Model loading time: 1.4021379947662354 seconds
Fetching time: 0.04068493843078613 seconds


## QPS Measurement

In [38]:
def measure_qps(QUERY, iterations=100):
    conn = create_db_connection() # Connect to the database
    conn.autocommit = True  # Enable autocommit for creating the database
    cursor = conn.cursor()
    start_time = time.time()
    for _ in range(iterations):
        cursor.execute(QUERY)  # Create the database
        cursor.fetchall()
    elapsed_time = time.time() - start_time
    qps = iterations / elapsed_time
    cursor.close()
    conn.close()

    return qps

### Generate Text Embedding for the Sample Text

In [5]:
sample_text = "summer outfit for thailand"
text_inputs = text_tokenizer(sample_text, return_tensors="pt", padding=True, truncation=True)
text_model_output = text_model(**text_inputs)
text_embeddings = text_model_output.last_hidden_state.mean(dim=1).squeeze().detach().cpu().numpy().tolist()

In [11]:
print(text_embeddings)

[0.09742061048746109, 0.49918133020401, -0.03261876106262207, 0.2645748257637024, 0.18288983404636383, 0.4575275778770447, 0.14676101505756378, 0.3150811493396759, -0.517536997795105, -0.17801591753959656, 0.2541957497596741, -0.5055475831031799, 0.0705580860376358, 0.4711122512817383, 0.6484673619270325, -0.19533346593379974, 0.261547327041626, 0.186905637383461, 0.3899439871311188, -0.4635244905948639, 0.21711668372154236, -0.07681413739919662, 0.25348740816116333, 0.12105589359998703, -0.03783831745386124, -0.5198584198951721, 0.48314517736434937, 0.4928768575191498, 0.07294569164514542, -0.21590955555438995, -0.2320374697446823, -0.021451549604535103, -0.22492942214012146, 0.2875055968761444, 0.15024526417255402, 0.23689620196819305, -0.3735382556915283, -0.08480574190616608, 0.07344142347574234, 0.6083762049674988, -0.23195263743400574, -0.2714826762676239, -0.19037440419197083, -0.11485058814287186, 0.2223883420228958, 0.4921382963657379, 0.0323888435959816, -0.23294684290885925,

## PGConf DE Experiments

### Partial Index on pgvector

In [ ]:
# QUERY = f"""SELECT img_id, 1-(embedding <-> '{text_embeddings}') AS score FROM benchmark_embeddings_pgvec ORDER BY embedding <-> '{text_embeddings}' LIMIT 10;"""
QUERY = f"""SELECT img_id,gender, productdisplayname , 1-(embeddings <-> '{text_embeddings}') AS score FROM product_vector WHERE gender='Men' ORDER BY score LIMIT 10""";
 
conn = create_db_connection() # Connect to the database
cursor = conn.cursor()


cursor.execute(QUERY)  # Create the database
cursor.fetchall()
# print(f"Queries Per Second (QPS): {qps:.2f}")

### QPS Measurement for pgvector

In [ ]:
# QUERY = f"""SELECT img_id, 1-(embedding <-> '{text_embeddings}') AS score FROM benchmark_embeddings_pgvec ORDER BY embedding <-> '{text_embeddings}' LIMIT 10;"""
QUERY = f"""SELECT img_id,gender, productdisplayname , 1-(embeddings <-> '{text_embeddings}') AS score FROM product_vector WHERE gender='Men' ORDER BY score LIMIT 10""";
 
# Run the benchmark
iterations = 100
qps = measure_qps(QUERY, iterations)
print(f"Queries Per Second (QPS): {qps:.2f}")

[('14170', 'Men', 'Belmonte Men Rough Finish Black Belts', -5.024000802028648),
 ('17031', 'Men', 'Gas Men Black Wave Flip Flops', -5.010693557505351),
 ('25794', 'Men', 'Arrow Men Navy Blue Suspenders', -4.990658163298269),
 ('49867', 'Men', 'Q&Q Men Black Digital Watch', -4.980545930470298),
 ('53598', 'Men', 'Q&Q Men Black Digital Watch', -4.980545930470298),
 ('44126', 'Men', 'Boss Men After Shave', -4.9755864960828715),
 ('55869',
  'Women',
  'Colorbar All Fired Up 2 Nail Lacquer 83V',
  -4.973074579651758),
 ('40779', 'Men', 'Quiksilver Men Brown Flip Flops', -4.973047756339448),
 ('38003', 'Men', 'Q&Q Men Black Watch', -4.969948213950109),
 ('17380',
  'Men',
  'Reid & Taylor Men Check Multi Muffler',
  -4.9677183847536375)]

In [ ]:
conn = create_db_connection() # Connect to the database
conn.autocommit = True  # Enable autocommit for creating the database
cursor = conn.cursor()

In [72]:
QUERY = f"""SELECT img_id, 1-(embedding <-> '{text_embeddings}') AS score FROM benchmark_embeddings_pgvec ORDER BY embedding <-> '{text_embeddings}' LIMIT 1000;"""
start_time = time.time()
cursor.execute(QUERY)  # Create the database
cursor.fetchall()
time.time() - start_time

0.007653951644897461

## Vchord

In [46]:
cursor.execute("""
        CREATE TABLE IF NOT EXISTS benchmark_embeddings_vchord2(
            img_id INTEGER PRIMARY KEY REFERENCES products(img_id) ON DELETE CASCADE,
            embedding vector(384));
    """)

cursor.execute("""CREATE INDEX ON benchmark_embeddings_vchord2 USING vchordrq (embedding vector_l2_ops) WITH (options = $$
residual_quantization = false
[build.internal]
lists = [1000]
spherical_centroids = true
$$);""")

InterfaceError: cursor already closed

In [ ]:
# Initialize timing variables for overall function performance tracking
function_start_time = time.time()

# Fetch data from the database
fetch_start = time.time()
cursor = conn.cursor()
cursor.execute("SELECT img_id, productdisplayname FROM products;")
result = cursor.fetchall()
fetch_end = time.time()

batch_size = 1000
total_rows_inserted = 0

for i in range(0, len(result), batch_size):
    batch_data = [(row[0], row[1]) for row in result[i:i+batch_size] if row[1]]
    if not batch_data:
        continue

    batch_ids, batch_texts = zip(*batch_data)
    batch_texts = [text for text in batch_texts if text]

    if not batch_texts:
        continue

    text_inputs = text_tokenizer(list(batch_texts), return_tensors="pt", padding=True, truncation=True)
    text_model_output = text_model(**text_inputs)
    text_embeddings = text_model_output.last_hidden_state.mean(dim=1).squeeze().detach().cpu().numpy().tolist()

    with conn.cursor() as cursor:
        for idx, embedding in enumerate(text_embeddings):
            cursor.execute(
                "INSERT INTO benchmark_embeddings_vchord2 (img_id, embedding) VALUES (%s, %s)",
                (batch_ids[idx], embedding)
            )
            total_rows_inserted += 1

function_end_time = time.time()
total_time = function_end_time - function_start_time

print(f"Total Rows: {total_rows_inserted}")
print(f"Total function execution time: {total_time} seconds")
print(f"Fetching time: {fetch_end - fetch_start} seconds")

# Total Rows: 44433
# Total function execution time: 249.79474711418152 seconds
# Embedding Insertion time: 248.98874711990356 seconds ~ 4 minutes
# Model loading time: 1.8063030242919922 seconds
# Fetching time: 0.04993581771850586 seconds

Total Rows: 44433
Total function execution time: 249.79474711418152 seconds
Model loading time: 1.8063030242919922 seconds
Fetching time: 0.04993581771850586 seconds


### QPS Vchord

In [75]:
QUERY = f"""SELECT img_id, 1-(embedding <-> '{text_embeddings}') AS score FROM benchmark_embeddings_vchord2 ORDER BY embedding <-> '{text_embeddings}' LIMIT 10;"""
# Run the benchmark
iterations = 100
qps = measure_qps(QUERY, iterations)
print(f"Queries Per Second (QPS): {qps:.2f}")

Queries Per Second (QPS): 540.76


In [73]:
QUERY = f"""SELECT img_id, 1-(embedding <-> '{text_embeddings}') AS score FROM benchmark_embeddings_vchord2 ORDER BY embedding <-> '{text_embeddings}' LIMIT 1000;"""
start_time = time.time()
cursor.execute(QUERY)  # Create the database
cursor.fetchall()
time.time() - start_time

0.01081395149230957

In [68]:
QUERY = f"""EXPLAIN ANALYZE SELECT img_id, 1-(embedding <-> '{text_embeddings}') AS score FROM benchmark_embeddings_vchord2 ORDER BY embedding <-> '{text_embeddings}' LIMIT 10;"""
cursor.execute(QUERY)  # Create the database
cursor.fetchall()

[('Limit  (cost=0.00..2.18 rows=10 width=20) (actual time=1.370..1.914 rows=10 loops=1)',),
 ('  ->  Index Scan using benchmark_embeddings_vchord2_embedding_idx on benchmark_embeddings_vchord2  (cost=0.00..9667.58 rows=44433 width=20) (actual time=1.367..1.888 rows=10 loops=1)',),
 ("        Order By: (embedding <-> '[0.09742063,0.4991813,-0.032618724,0.2645749,0.18288991,0.45752755,0.14676102,0.31508112,-0.51753694,-0.17801586,0.25419584,-0.50554764,0.070558235,0.47111228,0.6484674,-0.19533338,0.26154727,0.18690552,0.38994405,-0.46352467,0.21711674,-0.07681413,0.25348747,0.12105579,-0.037838425,-0.5198583,0.4831451,0.49287698,0.07294557,-0.21590959,-0.23203763,-0.021451745,-0.22492945,0.2875056,0.15024522,0.23689626,-0.37353808,-0.084805526,0.07344134,0.6083763,-0.23195252,-0.27148265,-0.19037423,-0.11485064,0.22238834,0.4921384,0.032388713,-0.232947,-0.46515298,0.47612762,0.50035065,-0.49673364,-0.31666833,-0.38258243,0.104849204,0.0826336,-0.042856835,0.16741274,-0.20582621,-0.44907

In [69]:
QUERY = f"""EXPLAIN ANALYZE SELECT img_id, 1-(embedding <-> '{text_embeddings}') AS score FROM benchmark_embeddings_pgvec ORDER BY embedding <-> '{text_embeddings}' LIMIT 10;"""
cursor.execute(QUERY)  # Create the database
cursor.fetchall()

[('Limit  (cost=72.60..80.78 rows=10 width=20) (actual time=1.579..1.607 rows=10 loops=1)',),
 ('  ->  Index Scan using benchmark_embeddings_pgvec_embedding_idx on benchmark_embeddings_pgvec  (cost=72.60..36398.18 rows=44433 width=20) (actual time=1.576..1.602 rows=10 loops=1)',),
 ("        Order By: (embedding <-> '[0.09742063,0.4991813,-0.032618724,0.2645749,0.18288991,0.45752755,0.14676102,0.31508112,-0.51753694,-0.17801586,0.25419584,-0.50554764,0.070558235,0.47111228,0.6484674,-0.19533338,0.26154727,0.18690552,0.38994405,-0.46352467,0.21711674,-0.07681413,0.25348747,0.12105579,-0.037838425,-0.5198583,0.4831451,0.49287698,0.07294557,-0.21590959,-0.23203763,-0.021451745,-0.22492945,0.2875056,0.15024522,0.23689626,-0.37353808,-0.084805526,0.07344134,0.6083763,-0.23195252,-0.27148265,-0.19037423,-0.11485064,0.22238834,0.4921384,0.032388713,-0.232947,-0.46515298,0.47612762,0.50035065,-0.49673364,-0.31666833,-0.38258243,0.104849204,0.0826336,-0.042856835,0.16741274,-0.20582621,-0.44907

Results for 'outfit for a beach party':

Querying similar catalog took 1.2464 seconds.

Number of elements retrieved: 10

Vchord outputs were very correct like bikinis and dresses

pgvector outputs were very similar too!

Results for 'outfit for galentine day':

Querying similar catalog took 1.1322 seconds.

Number of elements retrieved: 10
pgvector - vchord same output but vchord was faster!
Avirate Black Polka Dot Dress
Vero Moda Women Beige Dress
Elle Yellow Vintage Shift Dress

Results for 'men outfit for a valentine day':

Querying similar catalog took 1.1899 seconds.

Number of elements retrieved: 10

Vchord - pgvector same!
Park Avenue Men Cream Tie
HUGO Men Fragrance Gift Set
Parx Men Lavender Tie
Hakashi Men Lavender Silk Tie
Park Avenue Men Lavender Tie
Reid & Taylor Men Cream Tie
Park Avenue Men Lavender Tie
Playboy Men Pack of 2 Innerwear Vests
Reid & Taylor Men Cream Tie
Rasasi Men Pour Homme Perfume